# Proposal for Government Zonal Hospitals - Mumbai

## Introduction/Business Problem

Mumbai in India, stands at 6th place in the list of “Cities by population density” of the world. 
Traffic is also pretty bad there. Many a times ambulances are stuck in traffic and reaching to the nearest Government hospital becomes very difficult.
Keeping all this in mind, it would be better to recognise at least one hospital in a zone, where treatment can be given at subsidised cost.

To achieve this, 
1. Divide the city into Zones.
2. Consider Postal codes for City Division.
3. In additional to that, cluster the Postal codes into ‘K’ clusters.
4. Check if there is at least one hospital in every zone.
5. List out all the hospitals in the zone. This is to give options to pick the hospital for government affiliation.
6. If there is no hospital in the zone, check if there is at least one hospital in the clusters( to which this zone belongs to).
7. List at least one hospital, from the list of hospitals in the zone, to tag to the zones which does not have hospitals otherwise.
8. If there is not hospital even in the cluster, then that would indicate that there is a need for the hospital in that area. Suggest government to build at least one hospital in that zone/cluster.
9. Finally, there will be a list of areas and for every area there will be either a list of hospitals or at least one hospital that can be used for government association.

## Data section

Now, use foursquare API to find all the hospitals in every pincode using latlong

To get the postal codes of Mumbai, referred to https://worldpostalcode.com/india/maharashtra/mumbai/.

By web-Scraping extract area, pincode.

Using 'geolocator' find the latlong for pincode.

Cluster all the areas(using lat long) into an appropriate number of clusters by using KMeans.

With this latlong, use foursquare API to find the all the hospitals in the area of the Pincode.

For example : 
Area_name   PinCode   Latitude   Longitude
Agripada    400011    18.975302  72.824898

Check if any area does not have a hospital, in which case check if the cluster(to which this area belongs), has a hospital

Pick any hospital from the cluster hospital list and suggest to this area.

Finally, there will be either a list of hospital suggestions(in case of hospital in the zonal area) or atleast one hospital suggestion(in case of hospital in the cluster area)

## Methodology

My initial start up data is from a webpage https://worldpostalcode.com/india/maharashtra/mumbai/, which has the list of all postal codes in Mumbai along with the name of the areas. I did some webscrapping and extracted the data into a dataframe. 
We had 128 pincodes in total here. 

In [ ]:
    Area_name	        PinCode
0	A I Staff Colony	400029
1	Agripada	        400011
2	Airport	            400099
3	Ambewadi	        400004
4	Andheri East	    400069

Once this data was ready, I used Nomintatim to get the latlong. Here i obsereved that for some areas, the latlong could not found. I had to drop out those pincodes for which latlong could not be found. There were approximately 39 pincodes. Hence, the size of the dataframe reduced to 89 rows.
And then used Foursquare API to get the list of all Hospitals in that pincodes. Since there might be more than one hospital in one pincode, I created a list of all the hospitals in that pincode. The dataframe now is like -

In [ ]:
shape before: (128, 4)
shape after: (89, 4)

And then used Foursquare API to get the list of all Hospitals in that pincodes. Since there might be more than one hospital in one pincode, I created a list of all the hospitals in that pincode. The dataframe now is like -

In [ ]:

    Area_name	   PinCode	Latitude	Longitude	IsHospital	Hospital_list
0	Agripada	   400011	18.975302	72.824898	1	        [Wockhardt Hospital, Nair Hospital, Niar hospi...
1	Airport	       400099	19.090201	72.863808	0	        []
2	Ambewadi	   400004	19.186776	72.859313	1	        [st marys hospital]
3	Andheri East   400069	19.115883	72.854202	1	        [criticare hospital]
4	Andheri H.o	   400053	19.119125	72.847331	1	        [Sanjeevani Hospital, BSES Hospital Canteen, B...

As can be seen there are areas which does not have any hospitals. As a next step, i wanted to check if there is any other hospital nearby. To achive the same, i decided to cluster with the help of latlong(KMeans).
Before i decide, into how many clusters do i need to divide, i tried to find the optimum K. 

The optimum K was somewhere in between 10 and 20.Considering the vast spread city lets have 20 clusters let's cluster the city with k= 20.

I added the cluster numbers to the list as below -

In [ ]:
shape: (89, 7)
    Area_name	   PinCode	Latitude	Longitude	IsHospital	Hospital_list	                            clusters
0	Agripada	   400011	18.975302	72.824898	1	[Wockhardt Hospital, Nair Hospital, Niar hospi...	3
1	Airport	       400099	19.090201	72.863808	0	[]	                                                17
2	Ambewadi	   400004	19.186776	72.859313	1	[st marys hospital]	                                6
3	Andheri Eas    400069	19.115883	72.854202	1	[criticare hospital]	                            9
4	Andheri H.o	   400053	19.119125	72.847331	1	[Sanjeevani Hospital, BSES Hospital Canteen, B...	9

Then i created a folium map for these cluters

For all the pincodes that does not a hospital, i tried to check if there is a hospital in the cluster and pulled out the name of the hospital. And then added two more comlumns which indicate whether there is a hospital in the cluster and the name of hospital.

And finally i added a column, the name of the proposal hospital 'Govt_Hospital'. This contains the first name in the list of hospitals in the pincode. If there is no hospital in the pincode, it picks up the hospital name in the cluster hospital. And if there is no hospital in the pincode and also no hospital in the Cluster, then "No Hospital present" is written to the column

In [ ]:
    Area_name	 PinCode Latitude	Longitude	IsHospital	Hospital_list	                                    clusters cluster_Is_H    cluster_H_list	         Govt_Hospital
0	Agripada	 400011	 18.975302	72.824898	1	        [Wockhardt Hospital, Nair Hospital, Niar hospi...	 3	     0	             []	                     Wockhardt Hospital
1	Airport	     400099	 19.090201	72.863808	0	        []	                                                 17	No Hospital Present	 No Hospital Present	 No Hospital Present
2	Ambewadi	 400004	 19.186776	72.859313	1	        [st marys hospital]	                                 6	     0	             []	                     st marys hospital
3	Andheri East 400069	 19.115883	72.854202	1	        [criticare hospital]	                             9	     0	             []	                     criticare hospital
4	Andheri H.o	 400053	 19.119125	72.847331	1	        [Sanjeevani Hospital, BSES Hospital Canteen, B...	 9	     0	             []	                     Sanjeevani Hospital

## Results

So, the final result would be a dataframe which has the proposed Hospital for every pincode. This hospital name is extracted randomly from the list of hospitals in the pincode or randomly from the list of hospitals in the neighbourhood(in case where there are no hospitals in the pincode).

As an attempt to check the clustering, i have tried clustering it many times and each time i try to cluster the composition of the cluster was changing. But then, the number of clusters that don't have a hospital within the cluster remains more or less the same (3 or 4).

Also, while suggesting the hospital for government affiliation,for the ease of processing and the foursquare limitations, i could not use the ratings for each hospital. But i strongly feel that, ratings shud be taken into consideration(foursquare /venue/ ratings) and the highest rating hospital shud be marked. And only the highest rated hospitals shud be used for proposal for each area.


## Discussion

As, it can be seen there are areas(pincodes), where there is no hospital. Not only that, there are clusters(group of neighbourhoods) where there are no hospitals. This is a serious situation. Providing basic eminities like health care , for minimum charges should be the prime focus for any government.




There are many cases recorded, where in the patient died before even reaching the hospital.And in cities like Mumbai where the density of population is too high, traffic being bad, conjestion etc having a hospital in the near visinity is a must.

It was quite surprising for me to observe the fact that there are cluters that do not have hospitals. A very alarming fact.

I would strongly recommend to have a hospital in every cluster atleast, not to mention in each pincode. This is what that needs to be immediately acted upon.

## Conclusion

With all the observations and supporting data, i guess its high time we request to build hospitals in all those areas which don't have a hospital.

Let best healthcare be provided, lives be saved !!

Walk towards providing healthy future...

Sri.